Dhananjay Tiwari,
Ph.D. Mechanical Engineering,
Mechanical Engineering Laboratory, 1204
University of Illinois at Urbana Champaign

This file has the code for implementation of FLPO optimization problem using using scipy optimizers

In [53]:
# include libraries and dependencies
import numpy as np # Computational python programming and not include this? Can't happen!
import matplotlib.pyplot as plt # module to make plots
from scipy import optimize # optimization module in scipy
from scipy.optimize import minimize # minimize function using various algorithms
from scipy.optimize import Bounds # to define bounds on decision variables
from scipy.optimize import LinearConstraint # to define linear constraints on decision variables

In [64]:
# create FLPO class and define its associated functions
class FLPO():

    # define data members of the class
    N : int # number of vehicles
    M : int # number of nodes
    K : int # number of stages
    uav_id : list # vehicle indices
    node_id : list # facility indices
    stop_id : list # destination id
    node_param : dict # node parameters
    num_param : dict # number of parameters

    stages : list # FLPO stages, K elements
                  #   Nx1  (M+1)x1  (M+1)x1   (M+1)x1    (M+1)x1       1x1
                  #   stg0   stg1    stg2     stg3 . . . . stgK-2     stgK-1
                  #    0      0        0        0  . . . .  0               
                  #    1      1        1        1  . . . .  1
                  #    2      2        2        2  . . . .  2          dest
                  #    .      .        .        .  . . . .  .
                  #    .      .        .        .  . . . .  .
                  #    .      M-1      M-1      M-1  . . .  M-1
                  #    .      dest    dest     dest  . . .  dest
                  #    N-1    
    
    asn_pb : list # list of list of arrays
                  #  --------------                                                                           --------------
                  # |          stagewise probability associations for a vehicle i = 0,1, . . , N-1                          |
                  # |    -----                                                                     -----                    |
                  # |   |                                                                               |                   |
                  # |   |   stg0 ----> stg1 ----------> stg2 -----> . . . . ----> stg(K-1) -----> stgK  |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |  . . . . .        |
                  # |   |  1x|stg1|  |stg1|x|stg2|    |stg2|x|stg3|   . . . .    |stg(K-1)|x|stgK|      |                   |
                  # |   |  matrix     matrix            matrix  . . . . . . .      matrix               |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |    -----                                                                     -----                    |
                  #  -------------                                                                            --------------

    def __init__(self, N, M):
        # this function intializes the FLPO structure with assigned values
        
        # UAVs, nodes and stopping state indices
        self.N = N # initialize the number of vehicles
        self.M = M # initialize the number of nodes
        self.K = self.M + 2 # 1 initial stage + #intermediate stages equal to the number of facilities + 1 final stage
        self.uav_id = list(range(N)) # 0 1 2 . . . N-1
        self.node_id = list(range(N, N+M)) # N N+1 . . . N+M-1
        self.stop_id = list(range(N+M, N+M+1)) # N+M
        self.uav_init = list(range(N)) # # initialize the initial and terminal nodes of all the UAVs as zeros (to be given by the user)
        self.uav_stop = list(range(N)) # this value to be given by the user
        self.node_param = dict() # initialize dictionary to store facility parameters
        self.node_param['schedules'] = np.zeros([M, N]) # all M nodes have N components for each vehicle
        self.num_param = dict() # initialize dictionary to store the number of facility parameters
        self.num_param['schedules'] = M*N # the total numbe of parameters is number of nodes times the number of vehicles

        # define the structure of the stages and the elements in them
        self.stages = [0 for element in range(self.K)] # initialize list stages. Each 0 to be replaced with a column vector
        for stg in range(self.K):
            if stg == 0:
                # initial stage consisting of nodes
                self.stages[stg] = self.uav_id # Nx1 array
            elif stg == self.K-1:
                # final stage consists of only stopping state
                self.stages[stg] = self.stop_id # 1x1 array
            else:
                # other stages consist of facilities and destination
                self.stages[stg] = self.node_id + self.stop_id # (M+1) x 1 array
        
        # initialize association probabilities
        self.asn_pb = [[0 for element1 in range(self.K-1)] for element2 in range(N)]
        for n in range(N): # iteration over each node in the initial stage
            for stg in range(self.K-1): # iteration goes until the penultimate stage
                next_stage = self.stages[stg+1] # get the next stage nodes
                # initialize probabilities as ones (normalized in the next step)
                if stg == 0:
                    self.asn_pb[n][stg] = np.ones([1, len(next_stage)]) # uniform distribution
                    # self.asn_pb[n][stg] = np.random.rand(1, len(next_stage)) # normal distribution
                    self.asn_pb[n][stg] = self.asn_pb[n][stg]/self.asn_pb[n][stg].sum(axis = 1)[:, None] # normalize
                else:
                    self.asn_pb[n][stg] = np.ones([len(self.stages[stg]), len(self.stages[stg+1])]) # uniform distribution
                    # self.asn_pb[n][stg] = np.random.rand(len(self.stages[stg]), len(self.stages[stg+1])) # normal distribution
                    self.asn_pb[n][stg] = self.asn_pb[n][stg]/self.asn_pb[n][stg].sum(axis = 1)[:, None] # normalize

        print('\n -- FLPO Initialized -- ')

    # assign an initial node to a UAV
    def set_init_node(self, uav, init_node):
        id_uav = self.uav_id.index(uav) # get the UAV index
        self.uav_init[id_uav] = init_node # assign the initial node to the corresponding index
    
    # access the initial node of a UAV
    def get_init_node(self, uav):
        id_uav = self.uav_id.index(uav) # get the UAV index
        return self.uav_init[id_uav] # return the corresponding UAV initial node
    
    # assign a terminal node to a UAV
    def set_stop_node(self, uav, stop_node):
        id_uav = self.uav_id.index(uav) # get the UAV index
        self.uav_stop[id_uav] = stop_node # assign the terminal node to the corresponding index

    # access the terminal node of a UAV
    def get_stop_node(self, uav):
        id_uav = self.uav_id.index(uav) # get the UAV index
        return self.uav_stop[id_uav] # return the corresponding UAV terminal node

    # define stage transportation cost for each node        
    def cost_stage_transit(self, n, stg, i, j):
        # this function gives the value of stage transition cost for node n
        # from node i in stage stg to node j in stage stg+1
        # for now the cost is defined based on the facility schedules
        # print(' -- Data type of node parameters -- ' , type(self.node_param))
        t = self.node_param['schedules'] # a 2D matrix with each row representing a facility, and every column representing a node
        # get the indices of i and j and vehicle n
        id_i = self.stages[stg].index(i)
        id_j = self.stages[stg+1].index(j)
        id_n = self.uav_id.index(n)

        # define the constant for lower and upper time bound penalties respectively
        nu = 1
        eta = 1
        
        cost = 0
        if i in self.node_id and j in self.node_id:
            # time taken to move from node i to node j by vehicle n in intermediate nodes
            # the cost penalizes vehicles if they take a minimum of 30 second and a maximum of 80 second
            # to traverse between the nodes
            cost = t[id_j, id_n] - t[id_i, id_n] + \
                 np.exp(nu * (0.1 - (t[id_j, id_n] - t[id_i, id_n]))) + \
                    np.exp(eta * ((t[id_j, id_n] - t[id_i, id_n]) - 0.3))
            
        elif i in self.node_id and j in self.stop_id:
            # if transition is from a node to stopping state
            if i == self.get_stop_node(n):
                # if the current node is the destination of vehicle n then zero cost
                cost = 0
            else:
                # else high penalty
                cost = 100
        
        elif i in self.stop_id and j in self.node_id:
            # if transition is from a stopping state to a node
            cost = 100 # the penalty is high
        
        elif i in self.stop_id and j in self.stop_id:
            # zero cost for stage-transition from stop to stop
            cost = 0

        return cost
    

    # define the distortion function of a node 
    def distortion(self, n): 
        # this function computes the distortion function of a node n using probability associations and parameters of facilities for the corresponding vehicle
        if n not in self.uav_id: # check if the vehicle argument is valid or not
            print('invalid node id')
        else:
            # initialize the distortion data structure (same as stages data structure)
            D = [np.zeros(1)]
            D = D + [np.zeros(len(self.stages[stg])) for stg in range(1,self.K)]
            # print('\n-- Distortion Matrix --\n')
            # print(D)
            # for row in D:
            #     print(row)

            # iterate over stages : final stage --> initial stage
            stg_ids = list(range(self.K))
            stg_ids_rev = stg_ids[::-1]
            for stg in stg_ids_rev:
                # print('stage -- ' , stg)
                # iterate over each element of the chosen stage
                if stg > 0:
                    for i in self.stages[stg]:
                        id_i = self.stages[stg].index(i)
                        # print('  i -- ' , i, id_i)
                        D[stg][id_i] = 0
                        if stg == self.K-1:
                            D[stg][id_i] = 0
                        else:
                            for j in self.stages[stg+1]:
                                id_j = self.stages[stg+1].index(j)
                                # print('    j -- ' , j , id_j, D[stg][id_i])
                                D[stg][id_i] = D[stg][id_i] + self.asn_pb[n][stg][id_i, id_j] * (D[stg+1][id_j] + self.cost_stage_transit(n, stg, i, j))
                elif stg == 0:
                    i = n
                    id_i = 0
                    # print('  i -- ' , i , id_i)
                    for j in self.stages[stg+1]:
                        id_j = self.stages[stg+1].index(j)
                        # print( '    j -- ' , j, id_j, self.cost_stage_transit(n, stg, i, j))
                        D[stg][id_i] = D[stg][id_i] + self.asn_pb[n][stg][id_i, id_j] * (D[stg+1][id_j] + self.cost_stage_transit(n, stg, i, j))
        return D[0][0]
    

    def entropy(self, n): # depends on probability associations
        # ToDo
        return 0


    def free_energy():
        # ToDo
        return 0


    def flat_asn_pb(self, asn_pb):
        # this function unwraps the association probabilities into a column vector

        vec = []
        # unwrap for a vehicle, and then put them all together
        for uav_pb in asn_pb:
            for stg_pb in uav_pb:
                vec = vec + list(stg_pb.flatten())

        return vec


    def wrap_flat_asn_pb(self, vec_asn_pb):
        # this function wraps the association probabilities into their original defined structure inside __init__() function
        # vec_asn_pb (list) should contain probability association corresponding to all the vehicles otherwise the function prints error

        # initialize the required structure of association probability we need
        asn_pb = [[0 for element1 in range(self.K-1)] for element2 in range(self.N)]

        # get the dimension of probability association for a single vehicle
        dim_uav = 0
        # iterate over the stages to compute the above required dimension
        for stg in range(self.K-1):
            if stg == 0:
                dim_uav = dim_uav + len(self.stages[stg+1])
            else:
                dim_uav = dim_uav + len(self.stages[stg])*len(self.stages[stg+1])
        
        # dim_uav times the number of vehicles should be dim of vec_asn_pb
        if dim_uav * len(self.uav_id) != int(len(vec_asn_pb)):
            print('invalid dimension of association probability vector')
        else:
            for uav in self.uav_id:
                # get the index of uav
                n = self.uav_id.index(uav)
                # slice the probability vector for the corresponding vehicle
                sliced_vec = vec_asn_pb[n*dim_uav : (n+1)*dim_uav]
                k = 0
                for stg in range(self.K-1):
                    if stg == 0:
                        asn_pb[n][stg] = np.reshape( sliced_vec[k : k + 1*len(self.stages[stg+1])] , [1, len(self.stages[stg+1])] )
                        k = k + 1*len(self.stages[stg+1])
                    else:
                        asn_pb[n][stg] = np.reshape( sliced_vec[k : k + len(self.stages[stg])*len(self.stages[stg+1])] , [len(self.stages[stg]), len(self.stages[stg+1])] )
                        k = k + len(self.stages[stg])*len(self.stages[stg+1])
        
        return asn_pb


    def flat_schedules(self, schedules):
        # this converts the original data structure of schedules (see __init__() function) into a 1D array
        return list(schedules.flatten())


    def wrap_flat_schedules(self, vec_schedules):
        # this converts the 1D array of schedules into its original defined structure (see __init__() function)

        # first check if the dimension is correct
        dim = len(vec_schedules)
        # print('length of flattened schedules = ' , len(vec_schedules))
        # print('number of nodes = ' , len(self.node_id))
        # print('number of UAVs = ' , len(self.uav_id))
        if dim != len(self.node_id) * len(self.uav_id):
            print('invalid dimension of the parameters for time-schedule')

        schedules = np.reshape(vec_schedules, [len(self.node_id), len(self.uav_id)])

        return schedules
    

    def get_num_asn_pb(self):
        # this function outputs the number of probability association variables in the problem

        # get the number of probability association variables
        dim_pb = 0
        # iterate over the stages to compute the above required dimension
        for n in range(self.N):
            for stg in range(self.K-1):
                if stg == 0: # in the zeroth stage only 1 vehicle is considered and multiplied with the dimension of the stage 1
                    dim_pb = dim_pb + 1 * len(self.stages[stg+1])
                else: # otherwise the original dimensions of the current stage is multiplied with that the next one
                    dim_pb = dim_pb + len(self.stages[stg]) * len(self.stages[stg+1])

        return dim_pb
    
    
    def bounds_asn_pb(self):
        # this function returns the upper and lower bounds on the association variables
        # input : no external inputs
        # output : returns two vectors, representing lower and upper bounds respectively

        # fetch the number of association variables
        dim_pb = self.get_num_asn_pb()

        # associations lie in the interval [0, 1]
        lb = list(np.zeros(dim_pb))
        ub = list(np.ones(dim_pb))

        return lb, ub
    

    def bounds_schedules(self):
        # this functions returns the upper and lower bounds on the time-schedule parameters
        # input : no external input
        # output : returns two vectors, representing lower and upper bounds respectively

        # fetch the number of time parameters
        dim_schedules = self.num_param['schedules']

        # time variables are always non-negative
        lb = list(np.zeros(dim_schedules))
        ub = list(np.ones(dim_schedules)*np.inf)

        return lb, ub
    
    def bounds(self):
        # this function returns the upper and lower bounds on the decision variable consisting of associations and time-schedules
        # input : no external input
        # output : returns two vectors, representing lower and upper bounds respectively

        pb_lb, pb_ub = self.bounds_asn_pb() # fetch bounds on associations
        param_lb, param_ub = self.bounds_schedules() # fetch bounds on time-schedules

        # concatenate the bounds of the associations and the time-schedules
        lb = pb_lb + param_lb
        ub = pb_ub + param_ub
        print('lb' , lb)
        print('ub' , ub)
        bounds = Bounds(lb, ub)

        return bounds


    def cons_asn_pb(self):
        # this function constraints the distributions that the sum of probabilities at every node is 1 for every vehicle
        # input : no external input
        # output : returns a fat matrix resulting from the linear combinations from all the equations, 
        #          lower bounds and upper bounds

        # get the dimension of probability association for a single vehicle
        dim_pb = self.get_num_asn_pb() 

        # the number of equality constraints is equal to the sum of cardinality of every stage (except the first and the final stage)
        num_eq_cons = 0
        # iterate over the vehicles
        for n in range(self.N):
            # iterate till the penultimate stage
            for stg in range(self.K-1):
                if stg == 0: # in the initial stage, only vehicle n is counted
                    num_eq_cons = num_eq_cons + 1
                else: # otherwise count all the nodes in a stage
                    num_eq_cons = num_eq_cons + len(self.stages[stg])

        # initialize the linear constraint matrix
        fat_mat = [[0 for i in range(dim_pb)] for j in range(num_eq_cons)]
        # initialize counters
        row_count = 0 # to trace rows of fat_mat
        col_count = 0 # to trace cols of fat_mat
        # iterate over vehicles
        for n in range(self.N):
            # corresponding to the first transition of a vehicle
            # iterate over stages from 1 to K-1
            for stg in range(self.K-1):
                if stg == 0:
                    # iteration over stage 1 for columns of fat_mat in the first row
                    for j in range(len(self.stages[stg+1])):
                        fat_mat[row_count][col_count] = 1
                        col_count = col_count + 1 # update column count for the next column
                    row_count = row_count + 1 # update row count for the next row
                else:
                    # iterate over the rows that correspond to the current stage
                    for i in range(len(self.stages[stg])):
                        # iterate over the that cols correspond to the next stage
                        for j in range(len(self.stages[stg+1])):
                            fat_mat[row_count][col_count] = 1
                            col_count = col_count + 1 # update column count for the next column
                        row_count = row_count + 1 # update row count for the next row

        # define the lower bound for linear inequality/equality constraints
        lb = list(np.ones(num_eq_cons)) # all probability distributions sum to 1
        # define the upper bound for linear inquality/equality constraints (equal to lb in case of equality)
        ub = list(np.ones(num_eq_cons)) # all probability distributions sum to 1

        return fat_mat, lb, ub


    def cons_schedules(self):
        # this function gives the matrices corresponding constraints on the time-schedule parameters
        # input : no external inputs
        # output : matrices corresponding to linear combination of time schedules, upper bound and lower bound vectors

        # create matrix to compute the linear combinations of all the elements 
        fat_mat = list(np.eye(self.N * self.M)) # the time-schedules parameters should have N times M as its dimension
        for r in range(len(fat_mat)):
            fat_mat[r] = list(fat_mat[r]) # convert fat matrix into a list
        # create lower and upper bound arrays
        lb = [0 for el in range(self.N * self.M)]
        ub = [np.inf for el in range(self.N * self.M)]

        # get init nodes
        for n in self.uav_id:
            id_n = self.uav_id.index(n) # get the vehicle index
            init_node = self.get_init_node(n) # get the start node of the vehicle
            id_init = self.node_id.index(init_node) # get the index of the start node
            ub[id_init*self.N + id_n] = 0.0 # set the corresponding upper bound value equal to the start time
            lb[id_init*self.N + id_n] = 0.0 # set the corresponding lower bound value equal to the start time

        return fat_mat, lb, ub


    def cons(self):
        # this function defined all linear and nonlinear constraints for the system as per the requirements
        # input: no external inputs  
        # output: scipy.optimize_constraints methods for linear and nonlinear constraints

        # get the number of probability association variables
        dim_pb = self.get_num_asn_pb()
        # get the number of time schedule parameters
        dim_params = self.num_param['schedules']

        pb_mat, pb_lb, pb_ub = self.cons_asn_pb() # get the probability linear constraint matrices
        param_mat, param_lb, param_ub = self.cons_schedules() # get the schedule linear constraint matrices

        # fill the rows of probability matrix with zeros to accomodate for time-schedule variables 
        for i in range(len(pb_mat)):
            pb_mat[i] = pb_mat[i] + list(np.zeros(self.M*self.N)) # concatenation of lists
        # fill the rows of time-schedule parameter matrix to accomodate for association variables
        for j in range(len(param_mat)):
            param_mat[j] = list(np.zeros(dim_pb)) + param_mat[j] # concatenation of lists

        # concatenate both the matrices, upper bounds and lower bounds
        mat = pb_mat + param_mat
        lb = pb_lb + param_lb 
        ub = pb_ub + param_ub

        # define the linear constraint
        lincon = LinearConstraint(mat, lb, ub)

        return lincon #, non_lincon
    

    def objective(self, x):
        # this function computes the objective of the FLPO problem from distortion of each vehicle and entropy
        # of probability associations for each vehicle
        # input : x = [ flat_asn_pb , time_schedules ] is a vector consisting of all the probability associations and node time schedule parameters

        # get the number of probability association variables
        dim_pb = 0
        # iterate over the stages to compute the above required dimension
        for n in range(self.N):
            for stg in range(self.K-1):
                if stg == 0: # in the zeroth stage only 1 vehicle is considered and multiplied with the dimension of the stage 1
                    dim_pb = dim_pb + 1 * len(self.stages[stg+1])
                else: # otherwise the original dimensions of the current stage is multiplied with that the next one
                    dim_pb = dim_pb + len(self.stages[stg]) * len(self.stages[stg+1])
        # print('<< Inside objective() function >> \n << print the number of association probabilities >> ' , dim_pb)
        asn_pb = self.wrap_flat_asn_pb(x[0:dim_pb]) # probability association component of x
        params = self.wrap_flat_schedules(x[dim_pb:]) # time-schedules component of x
        
        # assign the association probabilities and node parameters 
        self.asn_pb = asn_pb
        self.node_param['schedules'] = params

        # initialize the total cost
        cost = 0

        # initialize the initial weight given to each vehicle
        uav_weights = np.ones(len(self.uav_id))/len(self.uav_id) # uniform distribution for now

        # compute distortion cost by performing weighted sum over each vehicle
        distortion_cost = 0
        for n in self.uav_id:
            id_n = self.uav_id.index(n) # get index
            distortion_cost = distortion_cost + uav_weights[id_n] * self.distortion(n) # update the weighed sum

        # ToDo : compute entropy cost by performing weighted sum over each vehicle
        
        # total cost
        cost = distortion_cost # + entropy_cost
        print(x, '--', cost)
        # print(' distortion cost = ' , cost)

        return cost

Minimize the distortion of FLPO with respect to probability associations and node schedule parameters

In [68]:
# initialize variables
N = 1 # number of vehicles
M = 2 # number of nodes

# define functions
def printMatrix(mat , mat_name):
    print(mat_name)
    for row in mat:
        print(row , '\n')
    return 0

# create an instance of FLPO class
flpo = FLPO(N, M)
print('\n -- FLPO Stages -- \n' , flpo.stages)
print('\n -- FLPO Associations -- \n' , flpo.asn_pb)


# set initial and stop points, and initial time for all the UAVs
init_nodes = [1]
stop_nodes = [2]
init_time = [0.0]
for i in range(N):
    flpo.set_init_node(flpo.uav_id[i], init_nodes[i])
    flpo.set_stop_node(flpo.uav_id[i], stop_nodes[i])
    flpo.node_param['schedules'][init_nodes[i], i] = init_time[i]

# get initial probability associations and time schedule parameters
vec_asn_pb = flpo.flat_asn_pb(flpo.asn_pb) # initial associations
vec_schedules = flpo.flat_schedules(flpo.node_param['schedules']) # initial schedules

x0 = vec_asn_pb + vec_schedules # initial decision variables
print('initial state' , x0)
# print(np.array(vec_asn_pb).shape)
cons = flpo.cons() # get the linear constraints
bounds = flpo.bounds() # get the bounds on decision variables
# print('\n -- The total dimension of decision variables = ' , len(x0) , '-- ')
res = minimize(flpo.objective, x0, method = 'trust-constr', constraints = cons, bounds = bounds, options={'disp': True})
# print(res)



 -- FLPO Initialized -- 

 -- FLPO Stages -- 
 [[0], [1, 2, 3], [1, 2, 3], [3]]

 -- FLPO Associations -- 
 [[array([[0.33333333, 0.33333333, 0.33333333]]), array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]]), array([[1.],
       [1.],
       [1.]])]]
initial state [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 1.0, 1.0, 1.0, 0.0, 0.0]
lb [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ub [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, inf, inf]
[0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 1.         1.         1.         0.         0.        ] -- 67.48710628

/Users/dhananjaytiwari/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  warn('Singular Jacobian matrix. Using SVD decomposition to ' +


In [67]:
#just a test block

# x = np.linspace(1.01, 1.99, 40)
# print(x)
# a = 4
# b = 4
# fx = x + np.exp(-a*(1 - x)) + np.exp(-b*(x - 2))
# # fx = x + 1/(x-1) + 1/(2 - x)

# plt.plot(x, fx)
# plt.show()
# print(res)

 barrier_parameter: 1.0240000000000006e-08
 barrier_tolerance: 1.0240000000000006e-08
          cg_niter: 306
      cg_stop_cond: 4
            constr: [array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 4.13550884e-16,
       1.32658786e+00]), array([4.46840378e-09, 9.99999994e-01, 1.50235185e-09, 7.55606120e-02,
       8.47780911e-01, 7.66584766e-02, 3.05185835e-10, 4.32378014e-08,
       9.99999956e-01, 4.89845937e-02, 1.68478348e-01, 7.82537059e-01,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 4.13550884e-16,
       1.32658786e+00])]
       constr_nfev: [0, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 4.440892098500626e-16
    execution_time: 0.460737943649292
               fun: 2.3664327630857093e-07
              grad: array([ 1.91012970e+01,  1.11261172e-07,  2.66447535e+01,  4.55089001e-07,
        1.97119334e-08,  4.46840378e-07,  1.0303